In [1]:
%cd ..

D:\Tesis\REPO\Cloud-Classification\code


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt

In [4]:
torch.manual_seed(7)
np.random.seed(7)

In [5]:
import os
from torchvision.io import read_image
import torchvision.transforms as T
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx
import glob

In [6]:
from src import config
from src import engine
from src import utils

In [7]:
from recipe.utils import *

Using backend: pytorch


In [8]:
torch.manual_seed(7)
np.random.seed(7)

### Dataloaders

In [9]:
def get_gcd_paths(data_dir, dataset_type):
    return glob.glob(
                     os.path.join(data_dir,f'GCD/{dataset_type}/**/*.jpg'), 
                     recursive=True
                    )

In [10]:
path_train_images = get_gcd_paths('D:\Tesis\REPO','train')
aug_images = get_gcd_paths('D:\Tesis\REPO','augmentation')

path_train_images+=aug_images

train_dataset = GCD(path_train_images, resize=224)

train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=4,
        num_workers=8,
        shuffle=True,
    )

In [11]:
len(train_dataset)

19848

In [12]:
path_test_images = get_gcd_paths('D:\Tesis\REPO','test')

test_dataset = GCD(path_test_images, resize=224)

test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=4,
        num_workers=8,
        shuffle=True,
    )

In [13]:
len(test_dataset)

9010

## Modelo

In [14]:
from ViG.src.models import IsotropicVIG

In [15]:
model = IsotropicVIG(in_channels=192, out_channels=7, num_blocks=12, kernel_size=16, patch_size=4)

In [16]:
from torchsummary import summary

In [17]:
summary(model, (3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
├─Patchifier: 1-1                        [-1, 192, 56, 56]         --
|    └─Conv2d: 2-1                       [-1, 192, 56, 56]         9,408
|    └─BatchNorm2d: 2-2                  [-1, 192, 56, 56]         384
├─ModuleList: 1                          []                        --
|    └─Block: 2-3                        [-1, 192, 56, 56]         --
|    |    └─Sequential: 3-1              [-1, 192, 56, 56]         221,760
|    └─Block: 2-4                        [-1, 192, 56, 56]         --
|    |    └─Sequential: 3-2              [-1, 192, 56, 56]         221,760
|    └─Block: 2-5                        [-1, 192, 56, 56]         --
|    |    └─Sequential: 3-3              [-1, 192, 56, 56]         221,760
|    └─Block: 2-6                        [-1, 192, 56, 56]         --
|    |    └─Sequential: 3-4              [-1, 192, 56, 56]         221,760
|    └─Block: 2-7                        [-1, 192, 56, 56]   

Layer (type:depth-idx)                   Output Shape              Param #
├─Patchifier: 1-1                        [-1, 192, 56, 56]         --
|    └─Conv2d: 2-1                       [-1, 192, 56, 56]         9,408
|    └─BatchNorm2d: 2-2                  [-1, 192, 56, 56]         384
├─ModuleList: 1                          []                        --
|    └─Block: 2-3                        [-1, 192, 56, 56]         --
|    |    └─Sequential: 3-1              [-1, 192, 56, 56]         221,760
|    └─Block: 2-4                        [-1, 192, 56, 56]         --
|    |    └─Sequential: 3-2              [-1, 192, 56, 56]         221,760
|    └─Block: 2-5                        [-1, 192, 56, 56]         --
|    |    └─Sequential: 3-3              [-1, 192, 56, 56]         221,760
|    └─Block: 2-6                        [-1, 192, 56, 56]         --
|    |    └─Sequential: 3-4              [-1, 192, 56, 56]         221,760
|    └─Block: 2-7                        [-1, 192, 56, 56]   

### Train con todo el batch

In [18]:
device = 'cuda'
EPOCHS = 20
LR = 3e-4

save_name = "graph_params_AUGMENTATION"

In [19]:
model = model.to(device)

In [20]:
from sklearn.metrics import accuracy_score

In [ ]:
model = model

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=0.9)


train_losses = []
test_losses = []


for e in range(EPOCHS):
    train_predictions, train_targets, train_loss = train(model, train_loader, criterion, optimizer, device)
    test_predictions, test_targets, test_loss = test(model, test_loader, criterion, device)
    
    train_acc = accuracy_score(train_targets, train_predictions)
    test_acc = accuracy_score(test_targets, test_predictions)
    
    print("EPOCH {}: Train acc: {:.2%} Train Loss: {:.4f} Test acc: {:.2%} Test Loss: {:.4f}".format(
        e+1,
        train_acc,
        train_loss,
        test_acc,
        test_loss
    ))
    
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    

  0%|                                                                               | 3/4962 [00:20<6:54:47,  5.02s/it]

In [ ]:
pd.Series(train_losses).plot.line()
pd.Series(test_losses).plot.line()
plt.show()